<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>LSTM - Clase 5 - V1.0</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt


In [5]:
# Lee el archivo como un DataFrame
data  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(data.head())
print(df_pid_validos.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [6]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

In [8]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
# En un nuevo Data Frame llamado "DATA"
data = data[data['product_id'].isin(product_ids_validos)]

In [ ]:

# Filtrar los datos para el período especificado
data['periodo'] = data['periodo'].astype(str)
data = data[(data['periodo'] >= '201801') & (data['periodo'] <= '201812')]

# Agrupar por product_id
products = data['product_id'].unique()

# Función para crear las secuencias de datos
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Diccionario para almacenar los resultados
results = {'product_id': [], 'tn': []}

for product_id in products:
    product_data = data[data['product_id'] == product_id]['tn'].values.reshape(-1, 1)
    
    # Estandarizar los datos
    scaler = StandardScaler()
    product_data = scaler.fit_transform(product_data)
    
    # Crear secuencias de datos
    seq_length = 12
    X, y = create_sequences(product_data, seq_length)
    
    # Dividir los datos en entrenamiento y prueba
    train_size = int(len(X) * 0.8)
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:], y[train_size:]
    
    # Crear el modelo LSTM
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1)
    
    # Realizar predicciones
    predictions = model.predict(X_test)
    
    # Revertir la estandarización
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test)
    
    # Almacenar los resultados
    results['product_id'].append(product_id)
    results['tn'].append(predictions[-1][0])

# Convertir los resultados a un DataFrame y guardarlos en un CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_tn.v1.csv', index=False)

Epoch 1/50


C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


659/659 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.8116
Epoch 2/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8759
Epoch 3/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6270
Epoch 4/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8708
Epoch 5/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8177
Epoch 6/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7898
Epoch 7/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6720
Epoch 8/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6103
Epoch 9/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.0174
Epoch 10/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6689
Epoch 11/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7373
Epoch 12/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.9737
Epoch 13/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5523
Epoch 14/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6924
Epoch 15/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8745


C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


916/916 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.9730
Epoch 2/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8652
Epoch 3/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9930
Epoch 4/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8682
Epoch 5/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6484
Epoch 6/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.7346
Epoch 7/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6963
Epoch 8/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9938
Epoch 9/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7826
Epoch 10/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9324
Epoch 11/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5911
Epoch 12/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.7697
Epoch 13/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8212
Epoch 14/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7715
Epoch 15/50
916/916 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.0121


...

1328/1328 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.5150
Epoch 2/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.9443
Epoch 3/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6742
Epoch 4/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.7652
Epoch 5/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.8399
Epoch 6/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.8372
Epoch 7/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.9331
Epoch 8/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8567
Epoch 9/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.0871
Epoch 10/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.8838
Epoch 11/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.9102
Epoch 12/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.9875
Epoch 13/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.9816
Epoch 14/50
1328/1328 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.6610
Epoch 15/50
1328/1328 ━━━━━━━━━━━━━━━━━━

C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.0093
Epoch 2/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.0487
Epoch 3/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.1716
Epoch 4/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.0521
Epoch 5/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8711
Epoch 6/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.1057
Epoch 7/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.2187
Epoch 8/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.0585
Epoch 9/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.0582
Epoch 10/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.4932
Epoch 11/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9154
Epoch 12/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.0315
Epoch 13/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.1576
Epoch 14/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8651
Epoch 15/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8277


C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


833/833 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.6159
Epoch 2/50
833/833 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.3234
Epoch 3/50
833/833 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8590
Epoch 4/50
833/833 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.1222
Epoch 5/50
833/833 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8870
Epoch 6/50
833/833 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.4930
Epoch 7/50
833/833 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.1544
Epoch 8/50
408/833 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.3748

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 